# DML Analysis

In this notebook, I will examine the location of differentially methylated loci (DMLs) in the *C. virginica* genome. The DMLs were identified using methylKit in [this R script](https://github.com/RobertsLab/project-virginica-oa/blob/master/analyses/2018-05-29-MethylKit-Full-Samples/2018-05-29-MethylKit-Analysis-Full-Samples.R). DMLs were then written out as a [bedfile](https://github.com/RobertsLab/project-virginica-oa/blob/master/analyses/2018-05-29-MethylKit-Full-Samples/2018-05-30-DML-Locations.bed). Using this file, I will begin the analysis derived from [Steven's  notebook](https://github.com/sr320/nb-2018/blob/master/C_virginica/21-Bedtools.ipynb).

1. Identify Overlaps with DMLs and Genomimc Feature Tracks
2. Calculate Overlap Proprtions
3. Gene Flanking
4. Enrichment Analysis

## 0. Set working directory

In [1]:
pwd

'/Users/yaamini/Documents/project-virginica-oa/notebooks'

In [2]:
cd ../analyses/

/Users/yaamini/Documents/project-virginica-oa/analyses


In [12]:
pwd

'/Users/yaamini/Documents/project-virginica-oa/analyses'

In [13]:
!mkdir 2018-06-11-DML-Analysis

In [14]:
ls

0516_bismark.err                     2018-05-04-Bismark-Full-Samples/
0516_bme.err                         2018-05-22-Bismark-Full-Samples/
0516_dedup.err                       2018-05-22-Bismark-Subset/
2018-01-23-MBDSeq-Labwork/           2018-05-29-MethylKit-Full-Samples/
2018-04-26-Gonad-Methylation-FastQC/ 2018-06-11-DML-Analysis/
2018-04-27-Bismark/                  README.md
2018-05-01-MethylKit/


In [3]:
cd 2018-06-11-DML-Analysis/

/Users/yaamini/Documents/project-virginica-oa/analyses/2018-06-11-DML-Analysis


## 1. Identify Overlaps with DMLs and Genomic Feature Tracks

To identify the location of DMLs in the *C. virginica* genome, I will use `intersect` from `bedtools`. [The BEDtools suite](http://bedtools.readthedocs.io/en/latest/content/bedtools-suite.html) allows me to easily find overlapping regions of different bed files.

### 1a. Locate BEDfiles for Analysis

The BEDfile with DMLs can be viewed below. Columns are are the chromosome, start position, end position, strand, and fold difference with direction. This file only has DMLs that were at least 50% different between the two treatments (control and elevated pCO2).

In [17]:
!head ../2018-05-29-MethylKit-Full-Samples/2018-05-30-DML-Locations.bed

NC_035780.1	265027	265029	-	63
NC_035780.1	346071	346073	-	54
NC_035780.1	549842	549844	+	-51
NC_035780.1	571093	571095	-	53
NC_035780.1	571138	571140	+	58
NC_035780.1	620088	620090	+	-52
NC_035780.1	635912	635914	-	-51
NC_035780.1	990995	990997	-	-50
NC_035780.1	993014	993016	+	-59
NC_035780.1	1887091	1887093	+	53


In [7]:
!wc -l ../2018-05-29-MethylKit-Full-Samples/2018-05-30-DML-Locations.bed

    1470 ../2018-05-29-MethylKit-Full-Samples/2018-05-30-DML-Locations.bed


I will be using the following Genome Feature Tracks:

1. Exon: Coding regions
2. Intron: Regions that are removed
3. mRNA: Code for proteins! The mRNA track includes both introns and exons.
4. CG motifs: Regions with CGs where methylation can occur

The links to these feature tracks can be found on the [Roberts Lab Genomic Resources wiki page](https://github.com/RobertsLab/resources/wiki/Genomic-Resources).

In [19]:
!curl http://eagle.fish.washington.edu/Cvirg_tracks/C_virginica-3.0_Gnomon_exon.bed > C_virginica-3.0_Gnomon_exon.bed

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 20.7M  100 20.7M    0     0  57.9M      0 --:--:-- --:--:-- --:--:-- 58.7M


In [22]:
!head C_virginica-3.0_Gnomon_exon.bed

NC_035780.1	13578	13603
NC_035780.1	14237	14290
NC_035780.1	14557	14594
NC_035780.1	28961	29073
NC_035780.1	30524	31557
NC_035780.1	31736	31887
NC_035780.1	31977	32565
NC_035780.1	32959	33324
NC_035780.1	66869	66897
NC_035780.1	64123	64334


In [8]:
!wc -l C_virginica-3.0_Gnomon_exon.bed

  731279 C_virginica-3.0_Gnomon_exon.bed


In [20]:
!curl http://eagle.fish.washington.edu/Cvirg_tracks/C_virginica-3.0_intron.bed > C_virginica-3.0_intron.bed

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9260k  100 9260k    0     0  37.9M      0 --:--:-- --:--:-- --:--:-- 38.4M


In [23]:
!head C_virginica-3.0_intron.bed

NC_035780.1	28961	28961
NC_035780.1	29074	30524
NC_035780.1	31558	31736
NC_035780.1	31888	31977
NC_035780.1	32566	32959
NC_035780.1	43110	43112
NC_035780.1	44359	45913
NC_035780.1	46507	64123
NC_035780.1	64335	66869
NC_035780.1	85606	85606


In [9]:
!wc -l C_virginica-3.0_intron.bed

  319262 C_virginica-3.0_intron.bed


In [18]:
!curl http://eagle.fish.washington.edu/Cvirg_tracks/C_virginica-3.0_Gnomon_mRNA.gff3 > C_virginica-3.0_Gnomon_mRNA.gff3

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 26.4M  100 26.4M    0     0  69.4M      0 --:--:-- --:--:-- --:--:-- 70.3M


In [24]:
!head C_virginica-3.0_Gnomon_mRNA.gff3

NC_035780.1	Gnomon	mRNA	28961	33324	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1
NC_035780.1	Gnomon	mRNA	43111	66897	.	-	.	ID=rna2;Parent=gene2;Dbxref=GeneID:111110729,Genbank:XM_022447324.1;Name=XM_022447324.1;gbkey=mRNA;gene=LOC111110729;model_evidence=Supporting evidence includes similarity to: 1 Protein%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments;product=FMRFamide receptor-like%2C transcript variant X1;transcript_id=XM_022447324.1
NC_035780.1	Gnomon	mRNA	43111	46506	.	-	.	ID=rna3;Parent=gene2;Dbxref=GeneID:111110729,Genbank:XM_022447333.1;Name=XM_022447333.1;gbkey=mRNA;gene=LOC111110729;model_evidence=Supporting

In [10]:
!wc -l C_virginica-3.0_Gnomon_mRNA.gff3

   60201 C_virginica-3.0_Gnomon_mRNA.gff3


In [21]:
!curl http://eagle.fish.washington.edu/Cvirg_tracks/C_virginica-3.0_CG-motif.bed > C_virginica-3.0_CG-motif.bed

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  533M  100  533M    0     0  52.4M      0  0:00:10  0:00:10 --:--:-- 48.8M


In [25]:
!head C_virginica-3.0_CG-motif.bed

NC_035780.1	28	30	CG_motif
NC_035780.1	54	56	CG_motif
NC_035780.1	75	77	CG_motif
NC_035780.1	93	95	CG_motif
NC_035780.1	103	105	CG_motif
NC_035780.1	116	118	CG_motif
NC_035780.1	134	136	CG_motif
NC_035780.1	159	161	CG_motif
NC_035780.1	209	211	CG_motif
NC_035780.1	224	226	CG_motif


In [11]:
!wc -l C_virginica-3.0_CG-motif.bed

 14458703 C_virginica-3.0_CG-motif.bed


### 1b. Use `intersect`

In [30]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed -h


Tool:    bedtools intersect (aka intersectBed)
Version: v2.26.0
Summary: Report overlaps between two feature files.

Usage:   bedtools intersect [OPTIONS] -a <bed/gff/vcf/bam> -b <bed/gff/vcf/bam>

	Note: -b may be followed with multiple databases and/or 
	wildcard (*) character(s). 
Options: 
	-wa	Write the original entry in A for each overlap.

	-wb	Write the original entry in B for each overlap.
		- Useful for knowing _what_ A overlaps. Restricted by -f and -r.

	-loj	Perform a "left outer join". That is, for each feature in A
		report each overlap with B.  If no overlaps are found, 
		report a NULL feature for B.

	-wo	Write the original A and B entries plus the number of base
		pairs of overlap between the two features.
		- Overlaps restricted by -f and -r.
		  Only A features with overlap are reported.

	-wao	Write the original A and B entries plus the number of base
		pairs of overlap between the two features.
		- Overlapping features restricted by -f and -r.
		  However, A fea

#### Exons

In [Steven's notebook](https://github.com/che625/olson-ms-nb/blob/master/.ipynb_checkpoints/BiGo_dev-checkpoint.ipynb), I noticed he said that there are some exon regions that do not code for any mRNA! The exon and mRNA regions need to be merged. I'm not sure how to do that, so I'll do what I can and then post an issue.

In [34]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-u \
-a ../2018-05-29-MethylKit-Full-Samples/2018-05-30-DML-Locations.bed \
-b C_virginica-3.0_Gnomon_exon.bed \
| wc -l
!echo "overlaps with exons"

     971
overlaps with exons


In [36]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-wb \
-a ../2018-05-29-MethylKit-Full-Samples/2018-05-30-DML-Locations.bed \
-b C_virginica-3.0_Gnomon_exon.bed \
> 2018-06-11-DML-Exon.txt

In [37]:
!head 2018-06-11-DML-Exon.txt

NC_035780.1	265027	265029	-	63	NC_035780.1	263244	265531
NC_035780.1	265027	265029	-	63	NC_035780.1	263245	265531
NC_035780.1	346071	346073	-	54	NC_035780.1	345983	346125
NC_035780.1	549842	549844	+	-51	NC_035780.1	549831	549992
NC_035780.1	571093	571095	-	53	NC_035780.1	570942	571194
NC_035780.1	571138	571140	+	58	NC_035780.1	570942	571194
NC_035780.1	620088	620090	+	-52	NC_035780.1	619970	620155
NC_035780.1	990995	990997	-	-50	NC_035780.1	990854	991062
NC_035780.1	993014	993016	+	-59	NC_035780.1	992913	993055
NC_035780.1	1887091	1887093	+	53	NC_035780.1	1887063	1887167


#### Introns

In [38]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-u \
-a ../2018-05-29-MethylKit-Full-Samples/2018-05-30-DML-Locations.bed \
-b C_virginica-3.0_intron.bed \
| wc -l
!echo "overlaps with introns"

     409
overlaps with introns


In [40]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-wb \
-a ../2018-05-29-MethylKit-Full-Samples/2018-05-30-DML-Locations.bed \
-b C_virginica-3.0_intron.bed \
> 2018-06-11-DML-Intron.txt

In [41]:
!head 2018-06-11-DML-Intron.txt

NC_035780.1	635912	635914	-	-51	NC_035780.1	635836	636202
NC_035780.1	2584492	2584494	+	57	NC_035780.1	2584154	2584505
NC_035780.1	2589699	2589701	+	-50	NC_035780.1	2589404	2589716
NC_035780.1	2689284	2689286	-	54	NC_035780.1	2679229	2698072
NC_035780.1	2689876	2689878	+	-51	NC_035780.1	2679229	2698072
NC_035780.1	4288213	4288215	+	-59	NC_035780.1	4288129	4288231
NC_035780.1	5803423	5803425	+	-56	NC_035780.1	5802914	5804722
NC_035780.1	7404772	7404774	+	-68	NC_035780.1	7403933	7405016
NC_035780.1	8833124	8833126	+	69	NC_035780.1	8832172	8833700
NC_035780.1	15513038	15513040	+	-62	NC_035780.1	15512246	15513167


#### mRNA

In [42]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-u \
-a ../2018-05-29-MethylKit-Full-Samples/2018-05-30-DML-Locations.bed \
-b C_virginica-3.0_Gnomon_mRNA.gff3 \
| wc -l
!echo "overlaps with mRNA"

    1353
overlaps with mRNA


In [43]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-wb \
-a ../2018-05-29-MethylKit-Full-Samples/2018-05-30-DML-Locations.bed \
-b C_virginica-3.0_Gnomon_mRNA.gff3 \
> 2018-06-11-DML-mRNA.txt

In [44]:
!head 2018-06-11-DML-mRNA.txt

NC_035780.1	265027	265029	-	63	NC_035780.1	Gnomon	mRNA	263244	272826	.	-	.	ID=rna22;Parent=gene17;Dbxref=GeneID:111124802,Genbank:XM_022468004.1;Name=XM_022468004.1;gbkey=mRNA;gene=LOC111124802;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 18 samples with support for all annotated introns;product=uncharacterized LOC111124802%2C transcript variant X2;transcript_id=XM_022468004.1
NC_035780.1	265027	265029	-	63	NC_035780.1	Gnomon	mRNA	263245	272839	.	-	.	ID=rna23;Parent=gene17;Dbxref=GeneID:111124802,Genbank:XM_022467995.1;Name=XM_022467995.1;gbkey=mRNA;gene=LOC111124802;model_evidence=Supporting evidence includes similarity to: 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 22 samples with support for all annotated introns;product=uncharacterized LOC111124802%2C transcript variant X1;transcript_id=XM_022467995.1
NC_035780.1	265027	265029	-	63	NC_035780.1	Gno

#### CG motifs

In [45]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-u \
-a ../2018-05-29-MethylKit-Full-Samples/2018-05-30-DML-Locations.bed \
-b C_virginica-3.0_CG-motif.bed \
| wc -l
!echo "overlaps with CGs"

    1470
overlaps with CGs


Proportion exon overlap with CG motifs:

In [19]:
1470/14458703

0.00010166887029908561

In [46]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-wb \
-a ../2018-05-29-MethylKit-Full-Samples/2018-05-30-DML-Locations.bed \
-b C_virginica-3.0_CG-motif.bed \
> 2018-06-11-DML-CGmotif.txt

In [47]:
!head 2018-06-11-DML-CGmotif.txt

NC_035780.1	265027	265029	-	63	NC_035780.1	265027	265029	CG_motif
NC_035780.1	346071	346073	-	54	NC_035780.1	346071	346073	CG_motif
NC_035780.1	549842	549844	+	-51	NC_035780.1	549842	549844	CG_motif
NC_035780.1	571093	571095	-	53	NC_035780.1	571093	571095	CG_motif
NC_035780.1	571138	571140	+	58	NC_035780.1	571138	571140	CG_motif
NC_035780.1	620088	620090	+	-52	NC_035780.1	620088	620090	CG_motif
NC_035780.1	635912	635914	-	-51	NC_035780.1	635912	635914	CG_motif
NC_035780.1	990995	990997	-	-50	NC_035780.1	990995	990997	CG_motif
NC_035780.1	993014	993016	+	-59	NC_035780.1	993014	993016	CG_motif
NC_035780.1	1887091	1887093	+	53	NC_035780.1	1887091	1887093	CG_motif


It's also useful to understand where the CG regions are in relation to exons, introns, and mRNA!

In [48]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-u \
-a C_virginica-3.0_Gnomon_exon.bed \
-b C_virginica-3.0_CG-motif.bed \
| wc -l
!echo "overlaps with exons"

  636270
overlaps with exons


Proportion exon overlap with CG motifs:

In [16]:
636270/14458703

0.04400602184027157

In [51]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-wb \
-a C_virginica-3.0_Gnomon_exon.bed \
-b C_virginica-3.0_CG-motif.bed \
> 2018-06-11-Exon-CGmotif.txt

In [52]:
!head 2018-06-11-Exon-CGmotif.txt

NC_035780.1	13597	13599	NC_035780.1	13597	13599	CG_motif
NC_035780.1	28992	28994	NC_035780.1	28992	28994	CG_motif
NC_035780.1	29001	29003	NC_035780.1	29001	29003	CG_motif
NC_035780.1	29028	29030	NC_035780.1	29028	29030	CG_motif
NC_035780.1	30539	30541	NC_035780.1	30539	30541	CG_motif
NC_035780.1	30574	30576	NC_035780.1	30574	30576	CG_motif
NC_035780.1	30602	30604	NC_035780.1	30602	30604	CG_motif
NC_035780.1	30676	30678	NC_035780.1	30676	30678	CG_motif
NC_035780.1	30695	30697	NC_035780.1	30695	30697	CG_motif
NC_035780.1	30723	30725	NC_035780.1	30723	30725	CG_motif


In [49]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-u \
-a C_virginica-3.0_intron.bed \
-b C_virginica-3.0_CG-motif.bed \
| wc -l
!echo "overlaps with introns"

  245500
overlaps with introns


Proportion intron overlap with CG motifs:

In [17]:
245500/14458703

0.016979392964915317

In [53]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-wb \
-a C_virginica-3.0_intron.bed \
-b C_virginica-3.0_CG-motif.bed \
> 2018-06-11-Intron-CGmotif.txt

In [54]:
!head 2018-06-11-Intron-CGmotif.txt

NC_035780.1	29180	29182	NC_035780.1	29180	29182	CG_motif
NC_035780.1	29203	29205	NC_035780.1	29203	29205	CG_motif
NC_035780.1	29221	29223	NC_035780.1	29221	29223	CG_motif
NC_035780.1	29295	29297	NC_035780.1	29295	29297	CG_motif
NC_035780.1	29323	29325	NC_035780.1	29323	29325	CG_motif
NC_035780.1	29326	29328	NC_035780.1	29326	29328	CG_motif
NC_035780.1	29412	29414	NC_035780.1	29412	29414	CG_motif
NC_035780.1	29452	29454	NC_035780.1	29452	29454	CG_motif
NC_035780.1	29672	29674	NC_035780.1	29672	29674	CG_motif
NC_035780.1	29758	29760	NC_035780.1	29758	29760	CG_motif


In [50]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-u \
-a C_virginica-3.0_Gnomon_mRNA.gff3 \
-b C_virginica-3.0_CG-motif.bed \
| wc -l
!echo "overlaps with mRNA"

   60195
overlaps with mRNA


Proportion mRNA overlap with CG motifs:

In [18]:
60195/14458703

0.004163236495002352

In [55]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-wb \
-a C_virginica-3.0_Gnomon_mRNA.gff3 \
-b C_virginica-3.0_CG-motif.bed \
> 2018-06-11-mRNA-CGmotif.txt

In [56]:
!head 2018-06-11-mRNA-CGmotif.txt

NC_035780.1	Gnomon	mRNA	28993	28994	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1	NC_035780.1	28992	28994	CG_motif
NC_035780.1	Gnomon	mRNA	29002	29003	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:111126949,Genbank:XM_022471938.1;Name=XM_022471938.1;gbkey=mRNA;gene=LOC111126949;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 21 samples with support for all annotated introns;product=UNC5C-like protein;transcript_id=XM_022471938.1	NC_035780.1	29001	29003	CG_motif
NC_035780.1	Gnomon	mRNA	29029	29030	.	+	.	ID=rna1;Parent=gene1;Dbxref=GeneID:1111269

## 2. Calculate Overlap Proportions

## 3. Gene Flanking

It's not enough to identify mRNA and DML intersections! I also need to know if DMLs and CG motifs overlap with regions that flank mRNA. These flanking regions could be promoters or transcription factors that could regulate these processes. To do this, I will use `flank` from `bedtools`.

1. Path to `flankBed`
2. -i Path to mRNA GFF file
3. -g Path to *C. virginica* "genome" file. `flankBed` requires the start and stop position of each genome (see [this issue](https://github.com/RobertsLab/resources/issues/294#issuecomment-397712928)). I created a file like in TextWrangler using [chromosome lengths from NCBI](https://www.ncbi.nlm.nih.gov/genome/gdv/browser/?context=genome&acc=GCF_002022765.2).
4. -b 1000: Add 1000 bp flanks to each end of the coding region

In [5]:
! /Users/Shared/bioinformatics/bedtools2/bin/flankBed \
-i C_virginica-3.0_Gnomon_mRNA.gff3 \
-g 2018-06-15-bedtools-Chromosome-Lengths.txt \
-b 1000 \
> 2018-06-15-mRNA-1000bp-Flanks.bed

Now I'll take the BEDfile I made and use it in `intersectBed` to find overlaps with DMLs and CG motifs! 

1. Path to `intersectBed`
2. -wb: Write output according to the second file
3. -a: Path to BEDfile created with flanks
4. -b: Specify either DML or CG motif file
5. ">" filename: Redirect output to a .txt file

In [6]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-wb \
-a 2018-06-15-mRNA-1000bp-Flanks.bed \
-b ../2018-05-29-MethylKit-Full-Samples/2018-05-30-DML-Locations.bed \
> 2018-06-19-mRNA-100bp-Flanks-DML.txt

In [8]:
! /Users/Shared/bioinformatics/bedtools2/bin/intersectBed \
-wb \
-a 2018-06-15-mRNA-1000bp-Flanks.bed \
-b C_virginica-3.0_CG-motif.bed \
> 2018-06-19-mRNA-100bp-Flanks-CGmotif.txt